In [9]:
from imblearn.metrics import geometric_mean_score
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import tree

from multi_imbalance.ensemble.mrbbagging import MRBBagging

# load sklearn built-in dataset
dataset = load_wine()
X, y = dataset.data, dataset.target

# initialization of MEBBagging object
mrbbagging = MRBBagging(30, tree.DecisionTreeClassifier())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train the MRBBagging classifier
mrbbagging.fit(X_train, y_train)

# make predictions
y_pred = mrbbagging.predict(X_test)
geometric_mean_score(y_test, y_pred, correction=0.001)

0.9772648059188251